# Un semplice modello linguistico

Costruiamo uno "N-gram Language Model" allenandolo sui testi di Sanremo. Partiamo dal modello più semplice e usiamo bigrammi e trigrammi.

Un'ottima guida per costruire un semplice modello del genere attraverso l'utilissima libreria [NLTK](https://www.nltk.org/) può essere consultata [qui](https://www.nltk.org/api/nltk.lm.html).

In [62]:
from nltk.corpus import PlaintextCorpusReader
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline
import nltk
import os

## Caricare il corpus

Per prima cosa carichiamo il corpus. Usiamo l'ottima libreria [NLTK]() per gestire i molti file in cui il corpus è diviso

In [63]:
root = os.path.expanduser('~/Documents/sync/data/Sanremo_testi')
os.path.isdir(root)


True

In [64]:
corpus_sanremo = PlaintextCorpusReader(root, r'.*\.txt')
len(corpus_sanremo.fileids())

244

Il corpus contiene 244 file.

Ora, da ciascuno di loro, estraiamo il testo ed eseguiamo alcune comuni operazioni di preparazione:

* eseguiamo il *sentence splitting* e la tokenizzaione (NLTK lo farà per noi)
* eliminiamo i segni di punteggiatura
* convertiamo tutte le maiuscole in minuscole

In [65]:
sents = []
for s in corpus_sanremo.sents():
    toks = [t.lower() for t in s if t.isalnum()]
    sents.append(toks)

Alcune statistiche di base sul nostro corpus

In [66]:
print(f'There are {len(sents)} sentences in the corpus; \nThere are {sum([len(s) for s in sents])} tokens in the corpus')


There are 9825 sentences in the corpus; 
There are 552556 tokens in the corpus


## Addestriamo il nostro modello

### Padding e bigrammi

`NLTK.lm` mette a nostra disposizione uno strumento molto utile che permette di fare 3 cose molto facilmente:

- inserire due token speciali (`<s>` e `</s>`) per l'inizio e fine di frase
- calcolare tutti gli $n$-grammi fino ad un massimo dato (es. 3 per unigrammi, bigrammi e trigrammi)
- definire il nostro vocabolario (l'insieme di tutte le forme usate)

In [67]:
train, vocab = padded_everygram_pipeline(3, sents)

### Addestriamo i modelli

Per il bigram model:

In [68]:
bi_lm = MLE(2)
bi_lm.fit(train, vocab)
print(bi_lm.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 20314 items>


In [69]:
train, vocab = padded_everygram_pipeline(3, sents)

lm = MLE(3)
lm.fit(train, vocab)
print(lm.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 20314 items>


## Esploriamo i modelli

### Contare i bigrammi (trigram)

In [70]:
lm.counts[['il', 'mio']].N()

767

In [71]:
lm.counts[['un', 'brutto']].most_common(5)

[('voto', 2), ('sogno', 1)]

In [72]:
lm.counts[['il', 'mio']].most_common(5)

[('cuore', 151), ('amore', 53), ('nome', 31), ('mondo', 30), ('destino', 21)]

In [73]:
lm.counts['amore']

3880

### Probabilità

In [74]:
gens = lm.generate(10, text_seed=['amore', 'mio'], random_seed=3)
print(' '.join(gens))

che non la smette più e a volte il cuore


Qual è la probabilità di una parola $W$ data una sequenza? Vediamo alcune probabilità del modello a trigrammi 

In [75]:
lm.score('amore', ['il', 'mio'])

0.06910039113428944

Ricordatevi che, secondo la MLE, la probabilità di un trigramma si calcola secondo la formula

$P(amore|il\;mio) = \frac{f(\text{il mio amore})}{f(\text{il mio })}$

Qual è la probabilità di "mio" se la parola che precede è "amore"?

Qual è la probabilità di "cuore" preceduto da "mio"?

In [76]:
lm.score('amore', ['il', 'mio'])

0.06910039113428944

La freq di "il mio" è:

In [77]:
f_il_mio = lm.counts[['il', 'mio']].N()
print(f_il_mio)

767


La freq di "il mio amore" è:

In [78]:
f_il_mio_amore = lm.counts[['il', 'mio']]['amore']
print(f_il_mio_amore)

53


In [79]:
53 / 767

0.06910039113428944

Quali sono le 5 parole più probabili dopo un dato bigramma?

In [80]:
def get_top_5_most_probable_words(lm, bigram):
    word_probs = {word: lm.score(word, bigram) for word in lm.vocab}
    return sorted(word_probs.items(), key=lambda x: x[1], reverse=True)[:5]

In [81]:
get_top_5_most_probable_words(lm, ['dammi', 'un'])

[('bacio', 0.6229508196721312),
 ('po', 0.04918032786885246),
 ('minuto', 0.04918032786885246),
 ('attimo', 0.03278688524590164),
 ('grande', 0.03278688524590164)]

## Generare testo!

Possiamo usare il nostro modello per generare del testo dato un contesto iniziale (es. un bigramma). Generare un testo sulla base della probabilità degli eventi si dice **sampling**. Più tecnicamente, **sampling** è il processo di estrarre (campionare) un token dalla distribuzione di probabilità predetta dal modello, dato un certo contesto. Estraiamo ("a caso", ma rispettando la distribuzione delle probabilità) un token dal suo contesto precedente. Ad esempio, per fare sampling dalla frase `il gatto dorme sul`, se il nostro modello avesse la seguente distribuzione di probabilità:
- `divano`: 0.4
- `cuscino`: 0.35
- `tavolo`: 0.1

"tireremmo una specie di dado truccato" (per così dire!) in cui "divano" ha il 40% di probabilità di essere scelto, "cuscino" il 35%, "tavolo" il 10%...

Possiamo fare sampling per generare testo a partire da un contesto di partenza. Nel caso del nostro trigram model, il contesto è un bigramma (che può anche comprendere il token speciale `<s>` per l'inizio di frase):

In [82]:
g = lm.generate(10, text_seed=['<s>', 'una'], random_seed=42)
print(" ".join(g))

musica antica nemmeno una risposta e scusa ma non c


## Un generatore più sofisticato

In [83]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_from_seed(model, seed_words, num_words=10, random_seed=42):
    """
    Generate text starting from a given seed word.
    
    :param model: An ngram language model from `nltk.lm.model`.
    :param seed_words: The initial words to start generation.
    :param num_words: Max number of words to generate.
    :param random_seed: Seed value for randomness.
    :return: A generated sentence as a string.
    """
    content = list(seed_words)  # Start with the seed word
    context = tuple(seed_words)  # Context for the bigram model

    for _ in range(num_words - 1):  # -1 because seed word is already added
        next_word = model.generate(1, text_seed=context, random_seed=random_seed)
        if next_word == '</s>':  # Stop at sentence end
            break
        content.append(next_word)
        context = (next_word,)  # Update context

    return detokenize(content)


In [84]:
# generate_from_seed(lm, ['il', 'mio'], num_words=10, random_seed=32)
generate_from_seed(lm, ['il', 'mio'], num_words=50, random_seed=32)

'il mio amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore amore'

In [85]:
lm.generate(10)

['mai',
 'pensato',
 'di',
 'lasciarti',
 'vivere',
 'ma',
 'poi',
 'ti',
 'fanno',
 'sincero']

### Ancora più sofisticato

Aumentiamo la randomizzazione

In [86]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
import random

detokenize = TreebankWordDetokenizer().detokenize

def generate_from_seed_trigram(model, seed_words, max_length=100, temperature=0.8):
    """
    Generate text from a trigram model with a stochastic selection mechanism to avoid loops.

    :param model: A trained trigram language model from `nltk.lm.model`.
    :param seed_words: A tuple or list containing exactly two seed words.
    :param max_length: Maximum number of words to generate (default: 100).
    :param temperature: A parameter that controls randomness (higher = more randomness).
    :return: A generated sentence as a string.
    """
    if not isinstance(seed_words, (list, tuple)) or len(seed_words) != 2:
        raise ValueError("seed_words must be a list or tuple containing exactly two words.")

    content = list(seed_words)  # Start with the two given words
    context = tuple(seed_words)  # Initialize context for the trigram model

    for _ in range(max_length - 2):  # -2 because we already have two words
        possible_next_words = list(model.context_counts(context).keys())
        
        if not possible_next_words:  # No next word found, stop generation
            break

        # Get probabilities of next words
        probabilities = [model.score(w, context) for w in possible_next_words]

        # Apply softmax-like temperature scaling
        probabilities = [p ** (1 / temperature) for p in probabilities]
        total_prob = sum(probabilities)
        probabilities = [p / total_prob for p in probabilities]

        # Sample next word based on probabilities
        next_word = random.choices(possible_next_words, weights=probabilities, k=1)[0]

        if next_word == '</s>':  # Stop at end-of-sentence token
            break

        content.append(next_word)
        context = (context[1], next_word)  # Update context with the last two words

    return detokenize(content)


In [87]:
print(generate_from_seed_trigram(lm, 
                         ['sei', 'mia'],
                         max_length=50,
                         temperature=1.3
                         ))

sei mia e portami via amore amore amore sta zitto non fiatare non franare di più se c è sei qui sei sempre stanco perché tu perseguitavi i cristiani e giustamente lui ti ricorderà il segreto dell amor non potrò scordare mai ma domani partirà se ne vanno e tu


<!-- ## Appendice: vedere le concordanze -->

In [88]:
from nltk.text import Text

conc = Text(corpus_sanremo.words())

In [89]:
conc.concordance(['moschee'], width=100, lines=40)

Displaying 4 of 4 matches:
 croce E chi prega sui tappeti Le chiese e le moschee l ’ Imàm e tutti i preti Ingressi separati de
 croce E chi prega sui tappeti Le chiese e le moschee l ’ Imàm e tutti i preti Ingressi separati de
 balalajka Lungo i giardini tra le croci e le moschee Il fiume va più nero della sera Oltre la torr
 croce , chi prega sui tappeti Le chiese e le moschee , gli imam e tutti i preti Ingressi separati 


In [90]:
lm.context_counts(('foooffa'))

FreqDist({})

In [91]:
lm.generate(random_seed=1)

'baciare'

In [92]:
from collections import Counter

# Known context: "voglio un"
print("Generating with seen context ('pieno', 'di'):")
generated_seen = [lm.generate(1, text_seed=("pieno", "di")) for _ in range(100)]
seen_counter = Counter(generated_seen)
print(seen_counter)

# OOV context: "voglio foooffa" (never seen in training)
print("\nGenerating with OOV context ('tazzina', 'di'):")
generated_oov = [lm.generate(1, text_seed=("tazzina", "di")) for _ in range(100)]
oov_counter = Counter(generated_oov)
print(oov_counter)

Generating with seen context ('pieno', 'di'):
Counter({'poesia': 12, 'vele': 10, 'guai': 8, 'bagliori': 7, 'foschia': 6, 'nutella': 6, 'illusioni': 6, 'pirati': 6, 'cuori': 5, 'domande': 4, 'sensi': 4, 'speranze': 4, 'emozioni': 4, 'brillanti': 4, 'sole': 4, 'promesse': 3, 'volanti': 2, 'noi': 2, 'note': 2, 'malinconia': 1})

Generating with OOV context ('tazzina', 'di'):
Counter({'me': 13, 'più': 12, 'un': 8, 'te': 5, 'cambiare': 3, 'una': 3, 'chi': 2, 'cosa': 1, 'imparare': 1, 'avere': 1, 'restare': 1, 'abbracciarti': 1, 'cuoio': 1, 'testa': 1, 'amare': 1, 'memoria': 1, 'proibito': 1, 'sfiorarti': 1, 'luce': 1, 'gridare': 1, 'volare': 1, 'conchiglie': 1, 'questa': 1, 'vedere': 1, 'darti': 1, 'che': 1, 'caldi': 1, 'materie': 1, 'camminare': 1, 'scatto': 1, 'notte': 1, 'nuovo': 1, 'sbagliare': 1, 'averti': 1, 'nirvana': 1, 'potertele': 1, 'quel': 1, 'non': 1, 'sera': 1, 'dio': 1, 'vento': 1, 'cantare': 1, 'stanlio': 1, 'voi': 1, 'andarmene': 1, 'perdere': 1, 'tosse': 1, 'decidere': 1, 

In [93]:
# Extract unigram counts manually
unigram_counts = {word: lm.counts[word] for word in lm.vocab}
total_unigrams = sum(unigram_counts.values())

# Convert to probabilities
unigram_probs = {word: count / total_unigrams for word, count in unigram_counts.items()}


In [94]:
unigram_probs['così']

0.0028858370955097187

In [95]:
lm.generate(num_words=1, text_seed=['tazzina', 'di'], random_seed=1)

'così'

In [96]:
lm.counts[('pieno', 'di')]

FreqDist({'guai': 2, 'poesia': 2, 'emozioni': 1, 'bagliori': 1, 'sole': 1, 'speranze': 1, 'vele': 1, 'cuori': 1, 'pirati': 1, 'foschia': 1, ...})

In [97]:
bi_lm.counts[('tazzina',)]

FreqDist({})

In [98]:
bi_lm.generate(num_words=1, text_seed=['tazzina'])

'pensato'

In [99]:
bi_lm.score('di', ['tazza'])

0.5

In [100]:
bi_lm.score('di', ['tazzina'])

0

In [101]:
bi_lm.counts[['tazza']]

FreqDist({'di': 1, 'con': 1})

In [102]:
29066**2

844832356